In [1]:
print("listing package versions... ")
import sys
print("python:", sys.version)
import pickle as pkl
import seaborn as sns
print("seaborn:", sns.__version__)
import matplotlib as mpl
import matplotlib.pyplot as plt
print("matplotlib:", mpl.__version__)
import numpy as np
print("numpy:", np.__version__)
import numba as nb
print("numba:", nb.__version__)
import scipy as sp
import scipy.integrate as sp1
import scipy.interpolate as sp2
import scipy.misc as sp3
import scipy.optimize as sp4
print("scipy:", sp.__version__)
import os
from pathlib import Path
%matplotlib inline

listing package versions... 
python: 3.7.0 (default, Jun 28 2018, 13:15:42) 
[GCC 7.2.0]
seaborn: 0.9.0
matplotlib: 2.2.3
numpy: 1.15.1
numba: 0.39.0
scipy: 1.1.0


In [2]:
mpl.rcParams['figure.dpi'] = 100
mpl.rcParams['figure.figsize'] = 6,3

sns.set_context("paper")
sns.set(font='serif')
sns.set_style("white", {
        "font.family": "serif",
        "font.serif": ["Times", "Palatino", "serif"]})

pardir=Path(os.getcwd()).parent.parent
datadir=str(pardir) +'/Data'

In [3]:
@nb.jit(nopython=True)
def kernel(band):
    return np.array([np.power(1-np.power((band-x)/band,2),3) for x in range(1,band+1)]
                    +[np.power(1-np.power(x/band,2),3) for x in range(1,band)])

In [12]:
fc=8
for industry in ["leg", "mil", "sci", "edu", "tec"]:
    print('industry=',industry)
    for weight in ['55','80']:
        print('weight=',weight)

        myw=int(weight)*10
        #fc is the scaling factor

        mydir=datadir+"/derive/"+industry+weight+"/"

        design='loglinear'
        addr=mydir+industry+weight+"j_curve_"+design+".pkl"
        curve2=pkl.load(open(addr, "rb"))

        x2=[]
        y2=[]
        ww2=[]

        for w in list(curve2.keys()):
            try:
                x2+=[curve2[w][0]]
                y2+=[curve2[w][1]]
                ww2+=[w]
            except:
                pass

        design='linear'
        addr=mydir+industry+weight+"j_curve_"+design+".pkl"
        curve1=pkl.load(open(addr, "rb"))

        x1=[]
        y1=[]
        ww1=[]


        for w in list(curve1.keys()):
            try: 
                x1+=[curve1[w][0]]
                y1+=[curve1[w][1]]
                ww1+=[w]
            except:
                pass

        l=len(x1)

        x1temp=np.zeros(l)
        y1temp=np.zeros(l)

        area=0
        band=15
        mykernel=kernel(band)

        for i in range(0,l):
            area=0
            for j in range(max(0,i-band),min(i+band,l)-1):
                x1temp[i]+=x1[j]*mykernel[band+j-i]
                y1temp[i]+=y1[j]*mykernel[band+j-i]
                area+=mykernel[band+j-i]
            x1temp[i]=x1temp[i]/area
            y1temp[i]=y1temp[i]/area

        l=len(x2)

        x2temp=np.zeros(l)
        y2temp=np.zeros(l)

        area=0
        band=5
        mykernel=kernel(band)

        for i in range(0,l):
            area=0
            for j in range(max(0,i-band),min(i+band,l)-1):
                x2temp[i]+=x2[j]*mykernel[band+j-i]
                y2temp[i]+=y2[j]*mykernel[band+j-i]
                area+=mykernel[band+j-i]
            x2temp[i]=x2temp[i]/area
            y2temp[i]=y2temp[i]/area

        import numpy as np
        from scipy.optimize import curve_fit

        x1data = np.array(y1)
        y1data = np.array(x1)
        x2data = np.array(y2)
        y2data = np.array(x2)

        def func(x, a,b,c,d):
            return a+b*x+c*(x**2)+d*(x**3)#+e*(x**4)+f*(x**5)+g*(x**6)

        popt2, pcov2 = curve_fit(func, x2data, y2data, p0=(1,1,1,0))

        X2=np.linspace(min(y2),max(y2),500)
        Y2=[func(x, *popt2) for x in X2]

        popt1, pcov1 = curve_fit(func, x1data, y1data, p0=(1,1,1,0))

        X1=np.linspace(min(y1),max(y1),500)
        Y1=[func(x, *popt1) for x in X1]


        plt.plot(x1, y1, 'ro', markersize=0.3, color= 'black')
        plt.plot(x2, y2, 'ro', markersize=0.3, color= 'black')
        plt.plot(x2temp, y2temp, '--', markersize=1, color= 'blue',label='log-linear')
        plt.plot(x1temp, y1temp, '-', markersize=1, color= 'green', label='linear')
        #plt.plot(Y1, X1, '--', markersize=0.5, color= 'green', label='linear')
        plt.plot(curve1[myw][0], curve1[myw][1], 'ro', markersize=6, color= 'red', label='default linear')
        plt.plot(curve2[1000][0], curve2[1000][1], 'ro', markersize=6, color= 'orange', label='price over quality')
        plt.plot(curve1[100][0], curve1[100][1], 's', markersize=8, color= 'white')
        plt.plot(curve1[945][0], curve1[945][1], 'D', markersize=7, color= 'white')
        plt.plot(curve1[100][0], curve1[100][1], 's', markersize=6, color= 'purple', label='pure quality auction')
        plt.plot(curve1[945][0], curve1[945][1], 'D', markersize=5, color= 'purple', label='pure price auction')
        #plt.plot(curve2[1000][0], curve2[1000][1], 'ro', markersize=5, color= 'orange', label='price over quality')
        #plt.plot(Y2, X2, '-', markersize=0.5, color= 'blue', label='log-linear')  

        print('distance to price-over-quality: x:',round(curve1[myw][0]-curve2[1000][0],2), ' y:', round(curve1[myw][1]-curve2[1000][1],2))
        print('horizontal distance to log-linear:', round(curve1[myw][0]-func(curve1[myw][1], *popt2),2))
        
        print('-'*25)
        

        #plt.xlabel('expected rebate')
        #plt.ylabel('expected quality')      


        plt.legend(title='Dataset: '+industry+'-'+weight, loc="lower left")
        plt.tight_layout()
        #plt.savefig('pdf/'+industry+weight+'_plot.pdf', bbox_inches='tight')
        #plt.savefig('png/'+industry+weight+'_plot.png', bbox_inches='tight')
        plt.savefig('eps/'+industry+weight+'_plot.eps', bbox_inches='tight')

        plt.clf()

industry= leg
weight= 55
distance to price-over-quality: x: 18.45  y: -5.6
horizontal distance to log-linear: 3.94
-------------------------
weight= 80
distance to price-over-quality: x: 14.5  y: -4.25
horizontal distance to log-linear: 2.21
-------------------------
industry= mil
weight= 55
distance to price-over-quality: x: 5.24  y: -1.14
horizontal distance to log-linear: 1.8
-------------------------
weight= 80
distance to price-over-quality: x: 9.72  y: -6.1
horizontal distance to log-linear: 0.41
-------------------------
industry= sci
weight= 55
distance to price-over-quality: x: 5.9  y: -1.04
horizontal distance to log-linear: 3.12
-------------------------
weight= 80
distance to price-over-quality: x: 27.59  y: -13.59
horizontal distance to log-linear: 1.16
-------------------------
industry= edu
weight= 55
distance to price-over-quality: x: 2.79  y: -0.28
horizontal distance to log-linear: 2.1
-------------------------
weight= 80
distance to price-over-quality: x: 26.42  y: -

<Figure size 600x300 with 0 Axes>